In [1]:
import ROOT #Start by importing ROOT
import numpy as np
ROOT.enableJSVis() #need this to make interactive ROOT plots
c1 = ROOT.TCanvas() #declare the ROOT canvas
ROOT.gSystem.Load("/usr/local/software/quadis/build/lib/libquadis.so")

Welcome to JupyROOT 6.10/00


0

In [2]:
infile = ROOT.TFile("/home/vasu/quadis/build/bin/proc_RMS1000.root")
tree = infile.Get("T2")
num_files = tree.GetEntries()
print num_files

50000


In [3]:
infile2 = ROOT.TFile("/home/vasu/PPC/PPC_SigGen/SignalLib_root/new_OutputRZperfect.root")
libtree = infile2.Get("T1")
nevent = ROOT.NEvent()
libtree.SetBranchAddress("NEvent",nevent)

rtrue = np.zeros(num_files)
ztrue = np.zeros(num_files)
ravg = np.zeros(num_files)
zavg = np.zeros(num_files)
R_new = np.zeros(num_files)
Z_new = np.zeros(num_files)
rbest = np.zeros(10)
zbest = np.zeros(10)

vector_chi = ROOT.std.vector('vector<double>')()
tree.SetBranchAddress("blrPulse_Chis", vector_chi)
vector_R = ROOT.std.vector('double')()
tree.SetBranchAddress("R", vector_R)
vector_Z = ROOT.std.vector('double')()
tree.SetBranchAddress("Z", vector_Z)
for i in range (num_files):
    tree.GetEntry(i)
    rtrue[i] = vector_R[0]
    ztrue[i] = vector_Z[0]
    ordered_index = np.argsort(vector_chi[0])
    for k in range (10):
        libtree.GetEntry(ordered_index[k])
        rbest[k] = nevent.GetPulseData(1).GetParameterByName("R")
        zbest[k] = nevent.GetPulseData(1).GetParameterByName("Z")
    ravg[i] = sum(rbest)/10
    zavg[i] = sum(zbest)/10
    R_new[i] = (rtrue[i]-ravg[i])
    Z_new[i] = (ztrue[i]-zavg[i])
    

In [ ]:
Rrms = []
Zrms = []
for i in range (num_files):
    if ((i%1000)==0):
        print i
        hr = ROOT.TH1D("hr","hr",67,-33,34)
        hz = ROOT.TH1D("hz","hz",103,-51,52)
        for j in range (1000):
            if (i<1000):
                hr.Fill(R_new[j])
                hz.Fill(Z_new[j])
            else:
                count = (i)+j
                hr.Fill(R_new[count])
                hz.Fill(Z_new[count])
        Rrms.append(hr.GetStdDev())
        Zrms.append(hz.GetStdDev())
    hr.Clear()
    hz.Clear()

In [ ]:
R0 = np.zeros(num_files/1000, dtype = 'int')
Z0 = np.zeros(num_files/1000, dtype = 'int')

for i in range (len(rtrue)):
    if ((i%1000)==0):
        R0[i/1000] = rtrue[i]
        Z0[i/1000] = ztrue[i]

In [ ]:
hrRMS = ROOT.TH2F("hrRMS","hrRMS",34,0,34,52,0,52)
hzRMS = ROOT.TH2F("hzRMS","hzRMS",34,0,34,52,0,52)
h1XR = hrRMS.GetXaxis().GetBinWidth(1)
h2YR = hrRMS.GetYaxis().GetBinWidth(1)
h1XZ = hzRMS.GetXaxis().GetBinWidth(1)
h2YZ = hzRMS.GetYaxis().GetBinWidth(1)
 
for i in range (len(R0)):
    hrRMS.SetBinContent(int(R0[i]/h1XR),int(Z0[i]/h2YR),Rrms[i])
    hzRMS.SetBinContent(int(R0[i]/h1XZ),int(Z0[i]/h2YZ),Zrms[i])
    

In [ ]:
hrRMS.Draw("colz")
c1.Draw()

In [ ]:
print Rrms

In [ ]:
hzRMS.Draw("colz")
c1.Draw()